In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, Dropout, Conv2D, MaxPooling2D, Flatten, Reshape, GaussianNoise
from tensorflow.keras.constraints import MinMaxNorm

First downloas mnist and svhn datasets:
- mnist: http://yann.lecun.com/exdb/mnist/
- svhn: http://ufldl.stanford.edu/housenumbers/

In [8]:
X = np.load("../datasets/digits/svhn_X.npy")
y = np.load("../datasets/digits/svhn_y.npy")

Xt = np.load("../datasets/digits/mnist_X.npy")
yt = np.load("../datasets/digits/mnist_y.npy")

# Normalize the data to get values between (0, 1)

In [2]:
def get_encoder(input_shape):
    inputs = Input(input_shape)
    modeled = Conv2D(32, 5, activation='relu')(inputs)
    modeled = MaxPooling2D(2, 2)(modeled)
    modeled = Conv2D(48, 5, activation='relu')(modeled)
    modeled = MaxPooling2D(2, 2)(modeled)
    modeled = Flatten()(modeled)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mse')
    return model

def get_task(input_shape, output_shape=(10,), activation="softmax", C=1.):
    inputs = Input(input_shape)
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(modeled)
    modeled = Dense(np.prod(output_shape), activation=activation)(modeled)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mse')
    return model

In [3]:
inputs = Input((28, 28, 1))
encoder = get_encoder((28, 28, 1))
task = get_task(encoder.output_shape[1:])

encoded = encoder(inputs)
tasked = task(encoded)

model = Model(inputs, tasked)
model.compile(loss="categorical_crossentropy", optimizer=Adam(0.001), metrics=["accuracy"])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
model (Model)                (None, 768)               39280     
_________________________________________________________________
model_1 (Model)              (None, 10)                88010     
Total params: 127,290
Trainable params: 127,290
Non-trainable params: 0
_________________________________________________________________


In [7]:
domain = "svhn"

X = X.reshape(X.shape + (1,))
one = OneHotEncoder(sparse=False)
y_cat = one.fit_transform(y.reshape(-1, 1))

In [9]:
np.random.seed(0)
tf.random.set_seed(0)
model.fit(X, y_cat, batch_size=128, epochs=30)

Train on 60000 samples
Epoch 1/30
60000/60000 [==============================] - 32s 539us/sample - loss: 0.6494 - accuracy: 0.7950
Epoch 2/30
60000/60000 [==============================] - 33s 543us/sample - loss: 0.2502 - accuracy: 0.9250
Epoch 3/30
60000/60000 [==============================] - 32s 528us/sample - loss: 0.1808 - accuracy: 0.9449
Epoch 4/30
60000/60000 [==============================] - 33s 542us/sample - loss: 0.1409 - accuracy: 0.9566
Epoch 5/30
60000/60000 [==============================] - 32s 537us/sample - loss: 0.1168 - accuracy: 0.9636
Epoch 6/30
60000/60000 [==============================] - 31s 524us/sample - loss: 0.0975 - accuracy: 0.9696
Epoch 7/30
60000/60000 [==============================] - 32s 530us/sample - loss: 0.0889 - accuracy: 0.9720
Epoch 8/30
60000/60000 [==============================] - 31s 518us/sample - loss: 0.0778 - accuracy: 0.9752
Epoch 9/30
60000/60000 [==============================] - 31s 521us/sample - loss: 0.0728 - accuracy: 0.9

In [10]:
np.save("datasets/digits_preprocessed/svhn_X.npy", encoder.predict(X))
np.save("datasets/digits_preprocessed/minst_X.npy", encoder.predict(Xt))

np.save("datasets/digits_preprocessed/svhn_y.npy", y)
np.save("datasets/digits_preprocessed/minst_y.npy", yt)